# Climate data
This code loads the `climetlab` package and sets the source and dataset to retrieve data from the Climate Data Store (CDS) for the ERA5 reanalysis dataset. The API configuration was performed following `https://cds.climate.copernicus.eu/api-how-to`.

`climetlab.load_source` ->  The `product_type` is set to `"reanalysis"`, and the `variable` is set to `"2m_temperature"` to retrieve temperature data at 2 meters above ground level, or `"total_precipitation"` represents the total precipitation rate and its units are meters per second (m s-1)
  The `year` is set to `2022`, and the `month` is set to a list containing `9`, `10`, and `11` to retrieve data for September, October, and November of 2022.
   The `day` and `time` are set to `"01"` and `"12:00"`, respectively, to retrieve data for the first day of each month at noon.
    The `area` parameter is set to `[38.72, -9.14, 38.72, -9.14]` to specify the coordinates for Lisbon.
    The data is then loaded into an xarray object using the `to_xarray()` method and printed to the screen.
    Is there anything else you would like to know?

In [24]:
import climetlab as cml
import xarray as xr

In [25]:
# Set the source and dataset
source = cml.load_source("cds", "reanalysis-era5-single-levels",
                         product_type="reanalysis",
                         variable="2m_temperature",
                         year=2022,
                         month=[9, 10, 11],
                         day="01",
                         time="12:00",
                         area=[38.72, -9.14, 38.72, -9.14])

# Load the data
data_temp = source.to_xarray()

'\nThis code loads the `climetlab` package and sets the source and dataset to retrieve data from the Climate Data Store (CDS) for the ERA5 reanalysis dataset.\n The `product_type` is set to `"reanalysis"`, and the `variable` is set to `"2m_temperature"` to retrieve temperature data at 2 meters above ground level.\n  The `year` is set to `2022`, and the `month` is set to a list containing `9`, `10`, and `11` to retrieve data for September, October, and November of 2022.\n   The `day` and `time` are set to `"01"` and `"12:00"`, respectively, to retrieve data for the first day of each month at noon.\n    The `area` parameter is set to `[38.72, -9.14, 38.72, -9.14]` to specify the coordinates for Lisbon.\n    The data is then loaded into an xarray object using the `to_xarray()` method and printed to the screen.\n    Is there anything else you would like to know?'

In [26]:
print(data_temp)

<xarray.Dataset>
Dimensions:     (number: 1, time: 3, step: 1, surface: 1, latitude: 1,
                 longitude: 1)
Coordinates:
  * number      (number) int32 0
  * time        (time) datetime64[ns] 2022-09-01T12:00:00 ... 2022-11-01T12:0...
  * step        (step) timedelta64[ns] 00:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 38.72
  * longitude   (longitude) float64 -9.14
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    t2m         (number, time, step, surface, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-04-11T22:27 GRIB to CDM+CF via cfgrib-0.9.1...


In [27]:
# total_precipitation represents the total precipitation rate and its units are meters per second (m s-1)
source = cml.load_source("cds", "reanalysis-era5-single-levels",
                         product_type="reanalysis",
                         variable="total_precipitation",
                         year=2022,
                         month=[9, 10, 11],
                         day="01",
                         time="12:00",
                         area=[38.72, -9.14, 38.72, -9.14])

# Load the data
data_precip = source.to_xarray()

In [28]:
print(data_precip)

<xarray.Dataset>
Dimensions:     (number: 1, time: 3, step: 1, surface: 1, latitude: 1,
                 longitude: 1)
Coordinates:
  * number      (number) int32 0
  * time        (time) datetime64[ns] 2022-09-01T06:00:00 ... 2022-11-01T06:0...
  * step        (step) timedelta64[ns] 06:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 38.72
  * longitude   (longitude) float64 -9.14
    valid_time  (time, step) datetime64[ns] ...
Data variables:
    tp          (number, time, step, surface, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-04-11T22:27 GRIB to CDM+CF via cfgrib-0.9.1...


In [29]:
# Merge the temperature and precipitation data
data_merged = xr.merge([data_temp, data_precip])

In [30]:
print(data_merged)

<xarray.Dataset>
Dimensions:     (number: 1, time: 6, step: 2, surface: 1, latitude: 1,
                 longitude: 1)
Coordinates:
  * number      (number) int32 0
  * time        (time) datetime64[ns] 2022-09-01T06:00:00 ... 2022-11-01T12:0...
  * step        (step) timedelta64[ns] 00:00:00 06:00:00
  * surface     (surface) float64 0.0
  * latitude    (latitude) float64 38.72
  * longitude   (longitude) float64 -9.14
    valid_time  (time, step) datetime64[ns] NaT 2022-09-01T12:00:00 ... NaT
Data variables:
    t2m         (number, time, step, surface, latitude, longitude) float32 na...
    tp          (number, time, step, surface, latitude, longitude) float32 na...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:      